# Example Noise

Here we demonstrate an example noise analysis

In [ ]:
from __future__ import division, print_function

import numpy as np
import pandas as pd

import glob, json, os
import matplotlib.pyplot as plt

from enterprise.pulsar import Pulsar
from enterprise.signals import parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const
from enterprise.signals import utils

from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
from corner import corner, quantile

%matplotlib inline

change directories and such to match your usage

In [ ]:
psrName = 'J2317+1439'
basedir = os.path.abspath('/Users/ptb/Projects/pulsar/11yr/bwm_results/dr2lite')

datadir = os.path.join(basedir, 'partim_bwm')  # par/tim location
outdir = os.path.join(basedir, psrName)  # chain file and other output here

parfile = os.path.join(datadir, '{}.par'.format(psrName))
timfile = os.path.join(datadir, '{}.tim'.format(psrName))

# read in pulsar

In [ ]:
psr = Pulsar(parfile, timfile)

In [ ]:
fig = plt.figure(figsize=(12,4))
ax = fig.add_subplot(111)
ax.errorbar(psr.toas/86400, psr.residuals*1e6, psr.toaerrs*1e6, fmt='.')
ax.set_xlabel(r"Time of Arrival (MJD)")
ax.set_ylabel(r"TOA residual ($\mu$s)")
ax.set_title(psrName);

since we stripped the **DM variation** model from the `.par` file, you might see some band seperation in the residuals

# setup enterprise model

### DM model declarations (not in `enterprise` base)

In [ ]:
# exponential decay for J1713 DM event
@signal_base.function
def exp_decay(toas, freqs, log10_Amp=-7, t0=54000, log10_tau=1.7):
    t0 *= const.day
    tau = 10**log10_tau * const.day
    wf = - 10**log10_Amp * np.heaviside(toas-t0, 1) * np.exp(-(toas-t0)/tau)
    return wf * (1400/freqs)**2

# linear interpolation basis in time with nu^-2 scaling (DM variations)
@signal_base.function
def linear_interp_basis_dm(toas, freqs, dt=30*86400):

    # get linear interpolation basis in time
    U, avetoas = utils.linear_interp_basis(toas, dt=dt)

    # scale with radio frequency
    Dm = (1400/freqs)**2

    return U * Dm[:, None], avetoas

# DMX-like signal with Gaussian prior
@signal_base.function
def dmx_ridge_prior(avetoas, log10_sigma=-7):
    sigma = 10**log10_sigma
    return sigma**2 * np.ones_like(avetoas)

### data selections for white noise models

In [ ]:
# define selection by observing backend
bke = selections.Selection(selections.by_backend)

# special selection for ECORR (only used for wideband NANOGrav data)
bke_NG = selections.Selection(selections.nanograv_backends)

### define parameters

In [ ]:
# white noise parameters
#efac = parameter.Uniform(0.5, 10.0)
efac = parameter.Normal(1.0, 0.1)
equad = parameter.Uniform(-10, -4)
ecorr = parameter.Uniform(-10, -4)

# red noise and DM parameters
log10_A = parameter.Uniform(-20, -11)
gamma = parameter.Uniform(0, 7)

# DM exponential parameters (for J1713 only)
t0 = parameter.Uniform(psr.toas.min()/86400, psr.toas.max()/86400)
log10_Amp = parameter.Uniform(-10, -2)
log10_tau = parameter.Uniform(np.log10(5), np.log10(500))

# DM variations -- DMX-like
log10_sigma = parameter.Uniform(-10, -4)

### define signals

In [ ]:
# white noise signals
ef = white_signals.MeasurementNoise(efac=efac, selection=bke)
eq = white_signals.EquadNoise(log10_equad=equad, selection=bke)
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=bke_NG)

# red noise signal
pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
rn = gp_signals.FourierBasisGP(pl, components=30)

# DMX-like signal
dm_window = 10 * const.day
dm_basis = linear_interp_basis_dm(dt=dm_window)
dm_prior = dmx_ridge_prior(log10_sigma=log10_sigma)
dm = gp_signals.BasisGP(dm_prior, dm_basis, name='dm')

# DM exponential model (J1713 only)
wf = exp_decay(log10_Amp=log10_Amp, t0=t0, log10_tau=log10_tau)
dmexp = deterministic_signals.Deterministic(wf, name='exp')

# timing model
tm = gp_signals.TimingModel(use_svd=True)

In [ ]:
# full model
mod = ef + eq + rn + dm + tm

if 'NANOGrav' in psr.flags['pta']:
    mod += ec
if psr.name == 'J1713+0747':
    mod += dmexp

# set up PTA of one
pta = signal_base.PTA([mod(psr)])

# the sampler

## jump proposals

In [ ]:
class JumpProposal(object):

    def __init__(self, pta, snames=None):
        """Set up some custom jump proposals"""
        self.params = pta.params
        self.pnames = pta.param_names
        self.npar = len(pta.params)
        self.ndim = sum(p.size or 1 for p in pta.params)

        # parameter map
        self.pmap = {}
        ct = 0
        for p in pta.params:
            size = p.size or 1
            self.pmap[str(p)] = slice(ct, ct+size)
            ct += size

        # parameter indices map
        self.pimap = {}
        for ct, p in enumerate(pta.param_names):
            self.pimap[p] = ct

        self.snames = {}
        for sc in pta._signalcollections:
            for signal in sc._signals:
                self.snames[signal.signal_name] = signal.params


    def draw_from_prior(self, x, iter, beta):
        """Prior draw.

        The function signature is specific to PTMCMCSampler.
        """

        q = x.copy()
        lqxy = 0

        # randomly choose parameter
        idx = np.random.randint(0, self.npar)

        # if vector parameter jump in random component
        param = self.params[idx]
        if param.size:
            idx2 = np.random.randint(0, param.size)
            q[self.pmap[str(param)]][idx2] = param.sample()[idx2]

        # scalar parameter
        else:
            q[idx] = param.sample()

        # forward-backward jump probability
        lqxy = param.get_logpdf(x[self.pmap[str(param)]]) - param.get_logpdf(q[self.pmap[str(param)]])

        return q, float(lqxy)

    def draw_from_red_prior(self, x, iter, beta):

        q = x.copy()
        lqxy = 0

        signal_name = 'red noise'

        # draw parameter from signal model
        param = np.random.choice(self.snames[signal_name])
        if param.size:
            idx2 = np.random.randint(0, param.size)
            q[self.pmap[str(param)]][idx2] = param.sample()[idx2]

        # scalar parameter
        else:
            q[self.pmap[str(param)]] = param.sample()

        # forward-backward jump probability
        lqxy = param.get_logpdf(x[self.pmap[str(param)]]) - param.get_logpdf(q[self.pmap[str(param)]])

        return q, float(lqxy)

    def draw_from_dm_prior(self, x, iter, beta):

        q = x.copy()
        lqxy = 0

        signal_name = 'dm'

        # draw parameter from signal model
        param = np.random.choice(self.snames[signal_name])
        if param.size:
            idx2 = np.random.randint(0, param.size)
            q[self.pmap[str(param)]][idx2] = param.sample()[idx2]

        # scalar parameter
        else:
            q[self.pmap[str(param)]] = param.sample()

        # forward-backward jump probability
        lqxy = param.get_logpdf(x[self.pmap[str(param)]]) - param.get_logpdf(q[self.pmap[str(param)]])

        return q, float(lqxy)

## sampling groups

In [ ]:
def get_global_parameters(pta):
    """Utility function for finding global parameters."""
    pars = []
    for sc in pta._signalcollections:
        pars.extend(sc.param_names)

    gpars = np.unique(list(filter(lambda x: pars.count(x)>1, pars)))
    ipars = np.array([p for p in pars if p not in gpars])

    return gpars, ipars


def get_parameter_groups(pta):
    """Utility function to get parameter groupings for sampling."""
    ndim = len(pta.param_names)
    groups = [range(0, ndim)]
    params = pta.param_names

    # get global and individual parameters
    gpars, ipars = get_global_parameters(pta)
    if any(gpars):
        groups.extend([[params.index(gp) for gp in gpars]])

    for sc in pta._signalcollections:
        for signal in sc._signals:
            ind = [params.index(p) for p in signal.param_names if p not in gpars]
            if ind:
                groups.extend([ind])

    return groups

## setup sampler

In [ ]:
groups = get_parameter_groups(pta)

In [ ]:
# search dimension
x0 = np.hstack(p.sample() for p in pta.params)
ndim = len(x0)

# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2)

sampler = ptmcmc(ndim,
                 pta.get_lnlikelihood, pta.get_lnprior,
                 cov,
                 groups=groups,
                 outDir=outdir,
                 resume=True,
                )

outfile = os.path.join(outdir, 'params.txt')
with open(outfile, 'w') as f:
    for pname in pta.param_names:
        f.write(pname+'\n')

print(outdir)

In [ ]:
jp = JumpProposal(pta)
sampler.addProposalToCycle(jp.draw_from_prior, 10)
sampler.addProposalToCycle(jp.draw_from_red_prior, 10)
sampler.addProposalToCycle(jp.draw_from_dm_prior, 10)

In [ ]:
params = {p.name: p.sample() for p in pta.params}

In [ ]:
pta.get_lnlikelihood(params)

In [ ]:
N = int(5.0e4)

sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )

# Post Processing

after the chain has gathered sufficient samples you may make some diagnostic plots and save a noisefile.

In [ ]:
utils.

In [ ]:
def trace_plot(chain, pars, cols=3, wid_per_col=6, aspect_r = 4/3):
    rows = len(pars)//cols
    if rows*cols < len(pars):
        rows += 1

    ax = []
    width = wid_per_col * cols
    height = wid_per_col * rows / aspect_r
    fig = plt.figure(figsize=(width, height))

    for pp, par in enumerate(pars):
        ax.append(fig.add_subplot(rows, cols, pp+1))
        ax[pp].plot(chain[:,pp]) #, label='')
        ax[pp].set_xlabel(par)
    return fig

def hist_plot(chain, pars, cols=3, bins=30):
    rows = len(pars)//cols
    if rows*cols < len(pars):
        rows += 1

    ax = []
    fig = plt.figure(figsize=(6*cols, 5*rows))

    for pp, par in enumerate(pars):
        ax.append(fig.add_subplot(rows, cols, pp+1))
        ax[pp].hist(chain[:,pp], bins=bins, normed=True, histtype='step')
        ax[pp].set_xlabel(par)
    return fig

In [ ]:
with open(os.path.join(outdir, 'params.txt'), 'r') as f:
    params = [line.rstrip() for line in f]

# WN params
par_WN = []
idx_WN = []
for pp, par in enumerate(params):
    if 'efac' in par or 'equad' in par or 'ecorr' in par:
        par_WN.append(par)
        idx_WN.append(pp)

# DM params
par_DM = []
idx_DM = []
for ii, par in enumerate(params):
    if '_dm' in par:
        par_DM.append(par)
        idx_DM.append(ii)

# RN params (always last 2)
par_RN = params[-2:]
idx_RN = []
for par in par_RN:
    idx_RN.append(params.index(par))

In [ ]:
chain_raw = pd.read_csv(os.path.join(outdir, 'chain_1.txt'),
                    sep='\t', dtype=float, header=None).values
len(chain_raw)

In [ ]:
burnfrac = 0.15
thin = 10

burn = int(burnfrac * len(chain_raw))

ch_WN = chain_raw[burn::thin, idx_WN]
ch_RN = chain_raw[burn::thin, idx_RN]
ch_DM = chain_raw[burn::thin, idx_DM]
ch_like = chain_raw[burn::thin, -4]

corL = acor(ch_like)[0]
corA = acor(ch_RN[:,1])[0]
N = len(ch_like)
print("N = {}, corL = {}, corA = {}".format(N, corL, corA))

In [ ]:
ch_plot = np.hstack((ch_RN, ch_like.reshape(len(ch_like),1))) # RN and Likelihood chains together
par_plot = par_RN + ['log_likelihood']
hist_plot(ch_plot, par_plot, cols=3);

In [ ]:
hist_plot(ch_WN, par_WN, cols=3);